In [ ]:
import os

os.environ["LANGSMITH_API_KEY"] = "ls__YOUR_KEY"   # REQUIRED
os.environ["LANGCHAIN_API_KEY"] = "ls__YOUR_KEY"   # optional fallback
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "multi-agent-pipeline-demo"


In [ ]:
import os
from langgraph.graph import StateGraph, END
from langchain_ollama import ChatOllama
from typing import TypedDict

# --------------- LANGSMITH ENABLED ----------------
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "multi-agent-pipeline-demo"
os.environ["LANGCHAIN_API_KEY"] = "<your-key>"

# --------------- MODEL ----------------------------
llm = ChatOllama(model="mistral")

# --------------- STATE ----------------------------
class PipelineState(TypedDict):
    task: str
    plan: str
    research: str
    code: str
    test_results: str

# --------------- AGENTS ----------------------------
def planner(state: PipelineState):
    out = llm.invoke(f"Break this task into 3 steps:\n{state['task']}")
    return {"plan": out.content}

def researcher(state: PipelineState):
    out = llm.invoke(f"Provide research for this plan:\n{state['plan']}")
    return {"research": out.content}

def coder(state: PipelineState):
    out = llm.invoke(f"Write Python code using:\n{state['research']}")
    return {"code": out.content}

def tester(state: PipelineState):
    out = llm.invoke(f"Test this code:\n{state['code']}")
    return {"test_results": out.content}

# --------------- GRAPH ----------------------------
g = StateGraph(PipelineState)

g.add_node("planner", planner)
g.add_node("researcher", researcher)
g.add_node("coder", coder)
g.add_node("tester", tester)

g.set_entry_point("planner")

g.add_edge("planner", "researcher")
g.add_edge("researcher", "coder")
g.add_edge("coder", "tester")
g.add_edge("tester", END)

app = g.compile()

# --------------- RUN ------------------------------
result = app.invoke({"task": "Build a basic calculator in Python"})
print(result)
